In [6]:
import json
from rouge_score import rouge_scorer, scoring
scorer = rouge_scorer.RougeScorer(rouge_types = ["rouge1", "rouge2", "rougeLsum"], use_stemmer=True)
import matplotlib.pyplot as plt

import numpy as np
import nltk
n = 16

In [3]:
with open('../../data/cnndm/test_data.json', encoding='utf-8') as f:
    test_data = json.load(f)

test_targets = []
for d in test_data:
    test_targets.append(d['target'])

with open('../../data/subcnndm/train_data.json', encoding='utf-8') as f:
    train_data = json.load(f)

train_targets = []
for d in train_data:
    train_targets.append(d['target'])

In [8]:
pred_dir = "../results/JGR-large-beamsearch/train/generated_predictions_16.txt"
targets = train_targets

preds = []
with open(pred_dir,encoding='utf-8') as f:
    for line in f.readlines():
       preds.append(line.strip())


preds = [pred.strip() for pred in preds]
targets = [target.strip() for target in targets]

preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
targets = ["\n".join(nltk.sent_tokenize(target)) for target in targets]

scores = []
for i,t in enumerate(targets):
    scores = []
    ps = preds[i * 16: (i+1)*16]
    score_this = []
    for j,p in enumerate(ps):
        s = scorer.score(t, p)
        score_this.append(s["rouge1"].fmeasure)

    scores.append(score_this)

scores = np.array(scores)

min_scores = np.min(scores, axis=1)
max_scores = np.max(scores, axis=1)

x = np.arange(len(targets))
plt.figure(figsize=(10,5))
plt.title("rouge1")
plt.xlabel("return size")
plt.ylabel("rouge1")
# plt.plot(x, x_data, label='groundtruth')
plt.plot(x, max_scores, label='max')
plt.plot(x, min_scores, label='min')
plt.legend()
plt.ylim(30, 60)
# x_ticks = np.arange(41)[::5]
# plt.xticks(x_ticks, x_ticks+1)
plt.grid()

KeyboardInterrupt: 